In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import os
print(os.listdir("input"))

['event_type.csv']


In [2]:
# Extract all ZIP files
import zipfile
import glob

for zip_file in glob.glob('*.zip'):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('input')
        print(f'Extracted {zip_file}')

Extracted event_type.csv.zip
Extracted log_feature.csv.zip
Extracted resource_type.csv.zip
Extracted sample_submission.csv.zip
Extracted severity_type.csv.zip
Extracted test.csv.zip
Extracted train.csv.zip


### Loading datasets

In [3]:
%%time
event_type=pd.read_csv("input/event_type.csv",on_bad_lines='skip')
train = pd.read_csv("input/train.csv")
severity_type = pd.read_csv("input/severity_type.csv")
log_feature = pd.read_csv("input/log_feature.csv")
test = pd.read_csv("input/test.csv")
resource_type = pd.read_csv("input/resource_type.csv",on_bad_lines='skip')
sample_submission = pd.read_csv("input/sample_submission.csv")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 212 ms


In [4]:
print("test",test.shape)
print("train",train.shape)

test (11171, 2)
train (7381, 3)


### Input datasets heads

In [5]:
print('test',test.head())
print('train',train.head(4))
print('sample_submission',sample_submission.head())
print('event_type',event_type.shape,event_type.head(2))
print('severity_type',severity_type.shape,severity_type.head(2))
print('log_feature',log_feature.shape,log_feature.head(2))
print('resource_type',resource_type.shape,resource_type.head(2))

test       id      location
0  11066  location 481
1  18000  location 962
2  16964  location 491
3   4795  location 532
4   3392  location 600
train       id      location  fault_severity
0  14121  location 118               1
1   9320   location 91               0
2  14394  location 152               1
3   8218  location 931               1
sample_submission       id  predict_0  predict_1  predict_2
0  11066          0          1          0
1  18000          0          1          0
2  16964          0          1          0
3   4795          0          1          0
4   3392          0          1          0
event_type (31170, 2)      id     event_type
0  6597  event_type 11
1  8011  event_type 15
severity_type (18552, 2)      id    severity_type
0  6597  severity_type 2
1  8011  severity_type 2
log_feature (58671, 3)      id log_feature  volume
0  6597  feature 68       6
1  8011  feature 68       7
resource_type (21076, 2)      id    resource_type
0  6597  resource_type 8
1  8011  reso

In [6]:
train.head()

,id,location,fault_severity
0,14121,location 118,1
1,9320,location 91,0
2,14394,location 152,1
3,8218,location 931,1
4,14804,location 120,0


## ** Data conversion**

In [7]:
event_type['id']=pd.to_numeric(event_type['id'],errors='coerce')
#converting object datatype into numeric

In [8]:
event_type.dtypes

id             int64
event_type    object
dtype: object

# **Training Preprocessing**

### **Merging**

In [9]:
train_eve_mer=train.merge(event_type.drop_duplicates(subset=['id']),on='id')

In [10]:
train_merge2=train_eve_mer.merge(severity_type.drop_duplicates(subset=['id']),how='left',on='id')

In [11]:
train_merge3=pd.merge(train_merge2,log_feature.drop_duplicates(subset=['id']),how='left',on='id')

In [12]:
train_merge4=pd.merge(train_merge3,resource_type.drop_duplicates(subset=['id']),how='left',on='id')

#### **Merged Training data**

In [13]:
train_merge4.head()

,id,location,fault_severity,event_type,severity_type,log_feature,volume,resource_type
0,14121,location 118,1,event_type 34,severity_type 2,feature 312,19,resource_type 2
1,9320,location 91,0,event_type 34,severity_type 2,feature 315,200,resource_type 2
2,14394,location 152,1,event_type 35,severity_type 2,feature 221,1,resource_type 2
3,8218,location 931,1,event_type 15,severity_type 1,feature 80,9,resource_type 8
4,14804,location 120,0,event_type 34,severity_type 1,feature 134,1,resource_type 2


In [14]:
train_merge4.dtypes

id                 int64
location          object
fault_severity     int64
event_type        object
severity_type     object
log_feature       object
volume             int64
resource_type     object
dtype: object

### **Checking for missing values**

In [15]:
train_merge4.isnull().sum()

id                0
location          0
fault_severity    0
event_type        0
severity_type     0
log_feature       0
volume            0
resource_type     0
dtype: int64

### **Finding categorical columns**

In [16]:
cat_col=list(set(train_merge4.columns)-set(train_merge4._get_numeric_data().columns))

### **Categorical conversion**

In [17]:
for i in range(len(cat_col)):
    train_merge4[cat_col[i]]=train_merge4[cat_col[i]].astype('category')

    

### **Label encoding**

In [18]:
le=preprocessing.LabelEncoder()
for i in range(len(cat_col)):
    train_merge4[cat_col[i]]=le.fit_transform(train_merge4[cat_col[i]])

In [19]:
train_merge4.drop(['id'],axis=1,inplace=True)

In [20]:
target=train_merge4[['fault_severity']]

In [21]:
train_merge4.drop(['fault_severity'],axis=1,inplace=True)

## modeling


In [22]:
rfc=RandomForestClassifier()

In [23]:
rfc.fit(train_merge4,target)

/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

##Tried XGBOOST classifier ,throwing error
from sklearn.model_selection import train_test_split
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(train_merge4, target, test_size=test_size, random_state=5)

import xgboost as xgb
dtrain = xgb.DMatrix(data=X_train,label=y_train)

params = {'learning_rate':0.1,
        'n_estimators':500,
        'num_class':3,
        'eval_metric':['mlogloss'],
        'missing':-1,
        'nthread':4}

num_round = 10000
bst = xgb.train(params, dtrain, num_round)

dtest = xgb.DMatrix(data=train_merge4,label=target)

n_classes =3
preds = bst.predict(dtest,n_classes)

pd.DataFrame(preds).head()

best_preds

import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])
#Calcul de précision
from sklearn.metrics import accuracy_score
print(accuracy_score(best_preds, target))

In [24]:
train_merge4.head()

,location,event_type,severity_type,log_feature,volume,resource_type
0,131,22,1,143,19,2
1,850,22,1,146,200,2
2,163,23,1,88,1,2
3,870,5,0,211,9,8
4,134,22,0,21,1,2


In [25]:
test.head()

,id,location
0,11066,location 481
1,18000,location 962
2,16964,location 491
3,4795,location 532
4,3392,location 600


## **TEST data preparation**

In [26]:
test.head()

,id,location
0,11066,location 481
1,18000,location 962
2,16964,location 491
3,4795,location 532
4,3392,location 600


In [27]:
test.shape

(11171, 2)

## ** Test data merging**

In [28]:
test_merge1=pd.merge(test,event_type.drop_duplicates(subset=['id']),how='left',on='id')

In [29]:
test_merge2=pd.merge(test_merge1,severity_type.drop_duplicates(subset=['id']),how='left',on='id')

In [30]:
test_merge3=pd.merge(test_merge2,log_feature.drop_duplicates(subset=['id']),how='left',on='id')

In [31]:
test_merge4=pd.merge(test_merge3,resource_type.drop_duplicates(subset=['id']),how='left',on='id')

In [32]:
test_merge4.head()

,id,location,event_type,severity_type,log_feature,volume,resource_type
0,11066,location 481,event_type 35,severity_type 2,feature 230,24,resource_type 2
1,18000,location 962,event_type 11,severity_type 1,feature 203,9,resource_type 8
2,16964,location 491,event_type 35,severity_type 2,feature 235,10,resource_type 2
3,4795,location 532,event_type 27,severity_type 5,feature 240,1,resource_type 3
4,3392,location 600,event_type 15,severity_type 2,feature 82,6,resource_type 8


In [33]:
cat_col

['location', 'log_feature', 'event_type', 'severity_type', 'resource_type']

In [34]:
test_merge4.dtypes

id                int64
location         object
event_type       object
severity_type    object
log_feature      object
volume            int64
resource_type    object
dtype: object

### **Categorical conversion **

In [35]:
for i in range(len(cat_col)):
    test_merge4[cat_col[i]]=test_merge4[cat_col[i]].astype('category')


### **Label encoding**

In [36]:
le2=preprocessing.LabelEncoder()
for i in range(len(cat_col)):
    test_merge4[cat_col[i]]=le2.fit_transform(test_merge4[cat_col[i]])

In [37]:
test_merge4.dtypes

id               int64
location         int32
event_type       int32
severity_type    int32
log_feature      int32
volume           int64
resource_type    int32
dtype: object

#### **Droping unique column**

In [38]:
test_merge4.drop(['id'],axis=1,inplace=True)

In [39]:
train_merge4.columns

Index(['location', 'event_type', 'severity_type', 'log_feature', 'volume',
       'resource_type'],
      dtype='object')

In [40]:
test_merge4.columns

Index(['location', 'event_type', 'severity_type', 'log_feature', 'volume',
       'resource_type'],
      dtype='object')

#### **Prediction **

In [41]:
predict_test=rfc.predict_proba(test_merge4)

In [42]:
pred_df=pd.DataFrame(predict_test,columns=['predict_0', 'predict_1', 'predict_2'])

In [43]:
submission=pd.concat([test[['id']],pred_df],axis=1)

### **Submission file generation**

In [45]:
# Round prediction columns to 2 decimal places
submission[['predict_0', 'predict_1', 'predict_2']] = submission[['predict_0', 'predict_1', 'predict_2']].round(2)

In [47]:
submission.to_csv('sub.csv',index=False,header=True,float_format='%.2f')